In [65]:
from pynput.keyboard import Controller as KeyCtrl
from pynput.keyboard import Key

physical_keyboard = KeyCtrl()
buffered_word = ""
current_layout = "en"  # 초기 설정
converted_positions = set()

In [66]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import pandas as pd
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt

# Define the allowed characters (26 lowercase + 26 uppercase)
ALPHABET = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
char_to_idx = {char: idx for idx, char in enumerate(ALPHABET)}

def one_hot_encode(letter):
    """Return a 52-dim one-hot vector for a given letter."""
    vec = np.zeros(len(ALPHABET), dtype=np.float32)
    if letter in char_to_idx:
        vec[char_to_idx[letter]] = 1.0
    return vec
class LanguageClassifier(nn.Module):
    def __init__(self, input_size=52, hidden_size=128, num_layers=1):
        super(LanguageClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, lengths):
        # Pack the padded sequence
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_out, (h_n, _) = self.lstm(packed)
        # h_n is of shape (num_layers, batch, hidden_size); we use the last layer’s hidden state
        h_last = h_n[-1]  # shape: (batch, hidden_size)
        out = self.fc(h_last)  # shape: (batch, 1)
        out = self.sigmoid(out)  # Output between 0 and 1
        return out


model = LanguageClassifier(input_size=52, hidden_size=128, num_layers=1)

try:
    state_dict = torch.load("AI/best_model.pt")
    load_info = model.load_state_dict(state_dict, strict=False) # Keep strict=False for compatibility info

    if load_info.missing_keys or load_info.unexpected_keys:
        print(f"State dict has missing or unexpected keys.")
    else:
        print("Model loaded successfully.")

except FileNotFoundError:
    print("Model file not found.")
    # Handle file not found (e.g., initialize model differently)

except Exception as e:
    print(f"Error loading model: {e}")
    # Handle other loading errors

else:
    print("Loading process complete.") # Optional success log


def predict_language(input_str):
    model.eval()
    seq = [one_hot_encode(ch) for ch in input_str if ch in char_to_idx]
    if not seq: return 0.5  # 기본값
    
    seq_tensor = torch.tensor(seq).unsqueeze(0)
    length = torch.tensor([len(seq)])
    
    with torch.no_grad():
        output = model(seq_tensor, length)
    return output.item()

Model loaded successfully.
Loading process complete.


In [67]:
import platform
import os
import ctypes


def switch_layout(target_lang):
    if platform.system() == 'Windows':
        # HKL_US = 0x04090409, HKL_Korean = 0x04120412
        lang_code = 0x04120412 if target_lang == "ko" else 0x04090409
        ctypes.windll.user32.ActivateKeyboardLayout(lang_code, 0)
    elif platform.system() == 'Darwin':
        src = """
        # macOS 입력 소스 전환 코드 (Objective-C)
        """
        os.system(f"osascript -e '{src}'")

In [68]:
def process_conversion(new_layout):
    global buffered_word, converted_positions
    
    if new_layout == "ko":
        converted = conv_en2ko(buffered_word)
    if new_layout == "en":
        converted = conv_ko2en(buffered_word)
    
    # 백스페이스 시뮬레이션
    physical_keyboard.press(Key.backspace)
    physical_keyboard.release(Key.backspace)
    
    # 변환 텍스트 입력
    physical_keyboard.type(converted)
    converted_positions.add(len(buffered_word))

In [69]:
import subprocess
import sys
import platform
from pynput import keyboard
from datetime import datetime

# OS 감지
IS_MAC = sys.platform == 'darwin'
IS_WIN = sys.platform.startswith('win')

print(sys.platform)

# OS별 키보드 전환 함수 구현
if IS_MAC:

    def switch_language():
        # Control 키와 Space 키를 조합하여 전환
        command = """
        osascript -e 'tell application "System Events"
            key down control
            delay 0.000
            key down space
            delay 0.000
            key up space
            delay 0.000
            key up control
        end tell'
        """
        subprocess.run(command, shell=True)



    def check_language_mode():
        # MacOS의 현재 입력 소스 확인
        command = "defaults read ~/Library/Preferences/com.apple.HIToolbox.plist AppleSelectedInputSources"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        input_source_data = result.stdout
        #print(input_source_data)

        # English (ABC) 레이아웃 체크
        if '"KeyboardLayout Name" = ABC' in input_source_data or 'KeyboardLayout Name = ABC' in input_source_data:
            return 'en'
        # Korean 레이아웃 체크
        elif '"Bundle ID" = "com.apple.inputmethod.Korean"' in input_source_data or 'Bundle ID = com.apple.inputmethod.Korean' in input_source_data:
            return 'ko'
        else:
            return 'unknown'
elif IS_WIN:
    def switch_language():
        # Windows 관련 함수는 추후 구현 (현재 비워둠)
        pass

    def check_language_mode():
        # Windows 관련 함수는 추후 구현
        return 'unknown'
else:
    def switch_language():
        pass

    def check_language_mode():
        return 'unknown'

darwin


In [70]:
def replace_buffred_word(new_word,current_layout):
    ko_new_word = new_word
    #-- backspace simulation -- 
    if current_layout == "ko":# It is needed to measure the length of the buffered word
        new_word = conv_en2ko(new_word)
        #print(new_word)
    needed_backspace = len(new_word)
    #print(needed_backspace)
    if IS_MAC:
        for _ in range(needed_backspace):
            physical_keyboard.press(Key.backspace)
            physical_keyboard.release(Key.backspace)
    if IS_WIN:
        for _ in range(needed_backspace):
            physical_keyboard.press(Key.backspace)
            physical_keyboard.release(Key.backspace)
    #-- replace the buffered word --
    if current_layout == "ko":
        print("ko New word")
        print(ko_new_word)
        replace_word = ko_new_word
        switch_layout("en")
        #print(replace_word)
    elif current_layout == "en":
        replace_word = conv_en2ko(new_word)
        switch_layout("ko")
        #print(replace_word)
    physical_keyboard.type(replace_word)
    return replace_word

In [71]:
from process.KoEnMapper import conv_en2ko, conv_ko2en

In [72]:
#current_layout = "en"
#replace_buffred_word("dlrjs gksrnrdj")

In [73]:
# 상수 및 전역 변수 설정
THRESHOLD_HIGH = 0.95  # 영어 전환 임계값
THRESHOLD_LOW = 0.05   # 한글 전환 임계값
buffered_word = ""

In [74]:
# 초기 OS의 입력 소스를 확인 (Mac의 경우)
initial_mode = check_language_mode()
if initial_mode in ['en', 'ko']:
    current_layout = initial_mode

# 로그 함수 (키 이벤트 등 기록)
log_file = 'key_event_log.txt'
def log_message(message):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(log_file, 'a', encoding='utf-8') as f:
        f.write(f"{timestamp} - {message}\n")

# 레이아웃 전환 시 호출되는 함수
def switch_layout(new_layout):
    global current_layout, buffered_word
    if new_layout == current_layout:
        return

    # 전환 전, 지금까지 입력된 내용을 변환 (예: 영어 -> 한글 또는 한글 -> 영어)
    if new_layout == "ko":
        converted_text = conv_en2ko(buffered_word)
        print(f"Buffer conversion (Eng->Kor): '{buffered_word}' -> '{converted_text}'")
    elif new_layout == "en":
        converted_text = conv_ko2en(buffered_word)
        print(f"Buffer conversion (Kor->Eng): '{buffered_word}' -> '{converted_text}'")

    # 버퍼 초기화 후 키보드 레이아웃 전환
    buffered_word = ""
    switch_language()
    current_layout = new_layout
    print(f"Keyboard layout switched to {current_layout}")

# 키보드 이벤트 처리 함수
def on_press(key):
    global buffered_word, current_layout

    check_language_mode()#update current langugage

    try:
        # backspace 입력 시 버퍼에서 마지막 문자 삭제
        if key == keyboard.Key.backspace:
            if buffered_word:
                try:
                    buffered_word = buffered_word[:-1]
                    #print(f"Backspace pressed. Updated buffer: '{buffered_word}'")
                    return True
                except:#already empty
                    return True
            

        # 일반 문자 입력 처리
        if hasattr(key, 'char') and key.char:
            char = key.char
            # 숫자, 알파벳, 기호 등 필요한 문자들만 처리
            if char.isalpha() or char.isdigit() or char in "!@#$%^&*()_+-=[]{}|;:'\",.<>/?`~":
                if (current_layout == "ko"):
                    char = conv_ko2en(char)#change to english all the time
                    #print(char)
                buffered_word += char
                #print(f"Key pressed: '{char}', Buffer: '{buffered_word}'")
                
                # LSTM 모델로 언어 예측
                prob = predict_language(buffered_word)
                #print(f"LSTM prediction probability: {prob:.5f} (Eng=1, Kor=0)")

                # 현재 레이아웃과 예측값에 따라 키보드 전환
                if current_layout == "en" and prob <= THRESHOLD_LOW:
                    print("Detected Korean Switching to Korean layout.")
                    print(buffered_word)
                    replace_buffred_word(buffered_word,current_layout)
                    
                    #print("switched to ko")
                elif current_layout == "ko" and prob >= THRESHOLD_HIGH:
                    print("Detected English. Switching to English layout.")
                    print(buffered_word)
                    replace_buffred_word(buffered_word,current_layout)
                    
                    #print("switched to en")
        else:# 특수키 입력 시 버퍼 초기화
            #print(f"Space/Enter pressed. Buffer cleared: '{buffered_word}'")
            buffered_word = ""
            return True
    except Exception as e:
        print(f"Error: {str(e)}")
        
    return True
    
def on_release(key):
    # ESC 키 입력 시 프로그램 종료
    if key == keyboard.Key.esc:
        print("ESC pressed. Exiting program.")
        return False
    return True

In [75]:
with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()

Detected English. Switching to English layout.
englis
ko New word
englis
Buffer conversion (Kor->Eng): 'englis' -> 'englis'
Keyboard layout switched to en
Detected Korean Switching to Korean layout.
gksr
Buffer conversion (Eng->Kor): 'gksr' -> '한ㄱ'
Keyboard layout switched to ko


KeyboardInterrupt: 